# Test training code for S2AE

In [46]:
import math
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import open3d as o3d
from scipy import spatial


import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
from tqdm.auto import tqdm

from average_meter import AverageMeter
from data_splitter import DataSplitter
from training_set import TrainingSetLidarSeg
from loss import L2Loss
from model import Model
from model_encode_decode_simple import ModelEncodeDecodeSimple
from sphere import Sphere
from visualize import Visualize
    
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Initialize some parameter

In [47]:
print(f"Initializing CUDA...")
torch.cuda.set_device(0)
torch.backends.cudnn.benchmark = True

print(f"Setting parameters...")
bandwidth = 100
n_features = 2
learning_rate = 4.5e-3
n_epochs = 1
batch_size = 1
num_workers = 32
descriptor_size = 256
net_input_size = 2 * bandwidth

print(f"Initializing data structures...")
net = ModelEncodeDecodeSimple(bandwidth).cuda()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
criterion = L2Loss(alpha=0.5, margin=0.2)
writer = SummaryWriter()
model_save = 'test_training_params.pkl'
print(f"All instances initialized.")

Initializing CUDA...
Setting parameters...
Initializing data structures...
All instances initialized.


## Load the dataset

In [9]:
export_ds = '/mnt/data/datasets/nuscenes/processed'
img_filename = f"{export_ds}/images.npy"
cloud_filename = f"{export_ds}/clouds.npy"
sem_clouds_filename = f"{export_ds}/sem_clouds.npy"

print(f"Loading from images from {img_filename}, clouds from {cloud_filename} and sem clouds from {sem_clouds_filename}")
img_features = np.load(img_filename)
cloud_features = np.load(cloud_filename)
sem_cloud_features = np.load(sem_clouds_filename)
print(f"Shape of images is {img_features.shape}, clouds is {cloud_features.shape} and sem clouds is {sem_cloud_features.shape}")

Loading from images from /mnt/data/datasets/nuscenes/processed/images.npy, clouds from /mnt/data/datasets/nuscenes/processed/clouds.npy and sem clouds from /mnt/data/datasets/nuscenes/processed/sem_clouds.npy
Shape of images is (10, 2, 200, 200), clouds is (10, 2, 200, 200) and sem clouds is (10, 2, 200, 200)


In [33]:
# Initialize the data loaders
train_set = TrainingSetLidarSeg(bandwidth, cloud_features, sem_cloud_features)
print(f"Total size of the training set: {len(train_set)}")
split = DataSplitter(train_set, False, test_train_split=1.0, shuffle=True)

# Split the data into train, val and optionally test
train_loader, val_loader, test_loader = split.get_split(
    batch_size=batch_size, num_workers=num_workers)
train_size = split.get_train_size()
val_size = split.get_val_size()
test_size = split.get_test_size()
print("Training size: ", train_size)
print("Validation size: ", val_size)
if test_size == 0:
    print('Test size is 0. Configured for external tests')
else:
    print("Testing size: ", test_size)

Total size of the training set: 10
Training size:  9
Validation size:  1
Test size is 0. Configured for external tests


In [34]:
def adjust_learning_rate_exp(optimizer, epoch_num, lr):
    decay_rate = 0.96
    new_lr = lr * math.pow(decay_rate, epoch_num)
    for param_group in optimizer.param_groups:
        param_group['lr'] = new_lr

    return new_lr

def accuracy(dista, distb):
    margin = 0
    pred = (dista - distb - margin).cpu().data
    acc = ((pred < 0).sum()).float() / dista.size(0)
    return acc

def train_lidarseg(net, criterion, optimizer, writer, epoch, n_iter, loss_, t0):
    train_dist_gt = AverageMeter()    
    net.train()
    for batch_idx, (cloud, lidarseg_gt) in enumerate(train_loader):
        cloud, lidarseg_gt = cloud.cuda().float(), lidarseg_gt.cuda().float()
        
        enc_dec_cloud = net(cloud)
        distance_cloud_gt, loss, loss_total = criterion(enc_dec_cloud, lidarseg_gt)
        #loss_embedd = embedded_a.norm(2) + embedded_p.norm(2) + embedded_n.norm(2)
        #loss = loss_triplet + 0.001 * loss_embedd

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_ += loss_total.item()

        train_dist_gt.update(distance_cloud_gt.cpu().data.numpy().sum())        
        writer.add_scalar('Train/Loss', loss, n_iter)
        writer.add_scalar('Train/Distance/GT', train_dist_gt.avg, n_iter)
        n_iter += 1

        if batch_idx % 5 == 4:
            t1 = time.time()
            print('[Epoch %d, Batch %4d] loss: %.8f time: %.5f lr: %.3e' %
                  (epoch + 1, batch_idx + 1, loss_ / 5, (t1 - t0) / 60, lr))
            t0 = t1
            loss_ = 0.0
    return n_iter

## Training Loop

In [50]:
abort = False
train_iter = 0
val_iter = 0
loss_ = 0.0
val_accs = AverageMeter()
print(f'Starting training using {n_epochs} epochs')
for epoch in tqdm(range(n_epochs)):
    lr = adjust_learning_rate_exp(optimizer, epoch_num=epoch, lr=learning_rate)
    t0 = time.time()

    train_iter = train_lidarseg(net, criterion, optimizer, writer, epoch, train_iter, loss_, t0)    
    #val_iter = validate(net, criterion, optimizer, writer, epoch, val_iter)
    writer.add_scalar('Train/lr', lr, epoch)

print("Training finished!")
torch.save(net.state_dict(), model_save)

# Test
print("Starting testing...")
torch.cuda.empty_cache()
#test(net, criterion, writer)
#print("Testing finished!")
writer.close()
print("Testing finished!")

Starting training using 1 epochs


[conv] x shape is torch.Size([900, 1, 2, 2]) and y shape is torch.Size([900, 2, 10, 2])
nspec = 35990
b_in = 30
ass = 35990
encoded x shape is torch.Size([1, 10, 60, 60, 60])
size of x: torch.Size([1, 10, 60, 60, 60]),  w torch.Size([60])
size of x reshaped torch.Size([36000, 60])
integrated x shape is torch.Size([1, 10, 60, 60])
[conv] x shape is torch.Size([900, 1, 10, 2]) and y shape is torch.Size([900, 10, 2, 2])
nspec = 35990
b_in = 30
ass = 35990
decoded x shape is torch.Size([1, 2, 200, 200, 200])
size of x: torch.Size([1, 2, 200, 200, 200]),  w torch.Size([200])
size of x reshaped torch.Size([80000, 200])
integrated x shape is torch.Size([1, 2, 200, 200])
[conv] x shape is torch.Size([900, 1, 2, 2]) and y shape is torch.Size([900, 2, 10, 2])
nspec = 35990
b_in = 30
ass = 35990
encoded x shape is torch.Size([1, 10, 60, 60, 60])
size of x: torch.Size([1, 10, 60, 60, 60]),  w torch.Size([60])
size of x reshaped torch.Size([36000, 60])
integrated x shape is torch.Size([1, 10, 60, 6